# LlamaIndexPDFLoader

This notebook provides a quick overview for getting started with `LLamaIndex` [document loader](https://python.langchain.com/docs/concepts/document_loaders). For detailed documentation of all DocumentLoader features and configurations head to the [API reference](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.LlamaIndexPDFLoader.html).


## Overview
### Integration details

| Class                                                                                                                                                     | Package | Local | Serializable | JS support|
|:----------------------------------------------------------------------------------------------------------------------------------------------------------| :--- | :---: | :---: |  :---: |
| [PDFMinerLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.LlamaIndexPDFLoader.html) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ❌ |   
   
---------   

### Loader features

|       Source        | Document Lazy Loading | Native Async Support | Extract Images | Extract Tables |
|:-------------------:| :---: | :---: | :---: |:---: |
| LlamaIndexPDFLoader | ✅ | ❌ | ✅ | ❌ |


## Setup

### Credentials

`LLAMA_CLOUD_API_KEY` are required to use `LlamaIndexPDFLoader`.

If you want to get automated best in-class tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [1]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

Install **langchain_community** and **pdfminer-six**.

In [2]:
%pip install -qU langchain_community pdfminer-six
%pip install -q ../../../../dist/patch_langchain_pdf_loader*.whl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Initialization

Now we can instantiate our model object and load documents:

In [1]:
import getpass
import os
import dotenv
dotenv.load_dotenv()

if "LLAMA_CLOUD_API_KEY" not in os.environ:
    os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("Enter your Llama cloud API key: ")


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from patch_langchain_community.document_loaders import LlamaIndexPDFLoader

file_path = "./example_data/layout-parser-paper.pdf"
loader = LlamaIndexPDFLoader(file_path)

## Load

In [4]:
docs = loader.load()
docs[0]

Document(metadata={'author': '', 'creationdate': '2021-06-22T01:27:10+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2021-06-22T01:27:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'producer': 'pdfTeX-1.40.21', 'subject': '', 'title': '', 'trapped': 'False', 'total_pages': 16, 'source': './example_data/layout-parser-paper.pdf'}, page_content='# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis\n\nZejiang Shen1, Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain Lee4, Jacob Carlson3, and Weining Li5\n\n1 Allen Institute for AI\n\nshannons@allenai.org\n\n2 Brown University\n\nruochen_zhang@brown.edu\n\n3 Harvard University\n\nmelissadell@fas.harvard.edu,\njacob_carlson@fas.harvard.edu\n\n4 University of Washington\n\nbcgl@cs.washington.edu\n\n5 University of Waterloo\n\nw422li@uwaterloo.ca\n\n# Abstract\n\nRecent advances in document image analysis (DIA) have 

In [5]:
import pprint

pprint.pp(docs[0].metadata)

{'author': '',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'creator': 'LaTeX with hyperref',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'producer': 'pdfTeX-1.40.21',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf'}


## Lazy Load


In [6]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        pages = []
len(pages)

1

In [7]:
print(pages[0].page_content[:100])
pprint.pp(pages[0].metadata)

# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis

Zejiang Shen1, Ru
{'author': '',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'creator': 'LaTeX with hyperref',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'producer': 'pdfTeX-1.40.21',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf'}


The metadata attribute contains at least the following keys:
- source
- page (if in mode *page*)
- total_page
- creationdate
- creator
- producer

Additional metadata are specific to each parser.
These pieces of information can be helpful (to categorize your PDFs for example).

## Splitting mode & custom pages delimiter

When loading the PDF file you can split it in two different ways:
- By page
- As a single text flow

By default PDFMinerLoader will split the PDF as a single text flow.

### Extract the PDF by page. Each page is extracted as a langchain Document object:

In [8]:
loader = LlamaIndexPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
)
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

16
{'author': '',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'creator': 'LaTeX with hyperref',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'producer': 'pdfTeX-1.40.21',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf',
 'file_name': './example_data/layout-parser-paper.pdf',
 'page': 0}


In this mode the pdf is split by pages and the resulting Documents metadata contains the page number. But in some cases we could want to process the pdf as a single text flow (so we don't cut some paragraphs in half). In this case you can use the *single* mode :

### Extract the whole PDF as a single langchain Document object:

In [9]:
loader = LlamaIndexPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
)
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

1
{'author': '',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'creator': 'LaTeX with hyperref',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'producer': 'pdfTeX-1.40.21',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf'}


Logically, in this mode, the ‘page_number’ metadata disappears. Here's how to clearly identify where pages end in the text flow :

### Add a custom *pages_delimitor* to identify where are ends of pages in *single* mode:

In [10]:
loader = LlamaIndexPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
    pages_delimitor="\n-------THIS IS A CUSTOM END OF PAGE-------\n",
)
docs = loader.load()
print(docs[0].page_content[:5780])

# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis

Zejiang Shen1, Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain Lee4, Jacob Carlson3, and Weining Li5

1 Allen Institute for AI

shannons@allenai.org

2 Brown University

ruochen_zhang@brown.edu

3 Harvard University

melissadell@fas.harvard.edu,
jacob_carlson@fas.harvard.edu

4 University of Washington

bcgl@cs.washington.edu

5 University of Waterloo

w422li@uwaterloo.ca

# Abstract

Recent advances in document image analysis (DIA) have been primarily driven by the application of neural networks. Ideally, research outcomes could be easily deployed in production and extended for further investigation. However, various factors like loosely organized codebases and sophisticated model configurations complicate the easy reuse of important innovations by a wide audience. Though there have been ongoing efforts to improve reusability and simplify deep learning (DL) model development in disciplines like 

This could simply be \n, or \f to clearly indicate a page change, or \<!-- PAGE BREAK --> for seamless injection in a Markdown viewer without a visual effect.

# Extract images from the PDF

You can extract images from your PDFs with a choice of three different solutions:
- rapidOCR (lightweight Optical Character Recognition tool)
- Tesseract (OCR tool with high precision)
- Multimodal language model

You can tune these functions to choose the output format of the extracted images among *html*, *markdown* or *text*

The result is inserted between the last and the second-to-last paragraphs of text of the page.

### Extract images from the PDF with rapidOCR:

In [11]:
%pip install -qU rapidocr-onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [12]:
# ⚠ WARNING: images_to_text is not implemented in LlamaIndex
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_text_with_rapidocr,
)

loader = LlamaIndexPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_text_with_rapidocr(format="html"),
)
docs = loader.load()

print(docs[5].page_content)

# 6 Z. Shen et al.

# 1 start5

|X-interval|(X1, V1)|(x1; VI)|(x2, V2)|3|
|---|---|---|---|---|
|Rectangle|Quadrilateral|1| | |
|2 1|(XZ,|(XL;| | |

# Coordinate

# Extra features

# BlockText

# BlockTypeReadingOrder

coordinatel
textblock1
textblock2
layout1
9
# A list of the layout elements

Fig. 2: The relationship between the three types of layout data structures.

Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such

Be careful, RapidOCR is designed to work with Chinese and English, not other languages.

### Extract images from the PDF with Tesseract:

In [13]:
%pip install -qU pytesseract

Note: you may need to restart the kernel to use updated packages.


In [15]:
# ⚠ WARNING: images_to_text is not implemented in LlamaIndex
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_text_with_tesseract,
)

loader = LlamaIndexPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_text_with_tesseract(format="text"),
)
docs = loader.load()
print(docs[5].page_content)

# 6 Z. Shen et al.

# 1 start5

|X-interval|(X1, V1)|(x1; VI)|(x2, V2)|3|
|---|---|---|---|---|
|Rectangle|Quadrilateral|1| | |
|2 1|(XZ,|(XL;| | |

# Coordinate

# Extra features

# BlockText

# BlockTypeReadingOrder

coordinatel
textblock1
textblock2
layout1
9
# A list of the layout elements

Fig. 2: The relationship between the three types of layout data structures.

Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such

### Extract images from the PDF with multimodal model:

In [16]:
%pip install -qU langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key =")

In [19]:
# ⚠ WARNING: images_to_text is not implemented in LlamaIndex
from langchain_openai import ChatOpenAI
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_description,
)

loader = LlamaIndexPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_description(
        model=ChatOpenAI(model="gpt-4o", max_tokens=1024), format="markdown"
    ),
)
docs = loader.load()
print(docs[5].page_content)

# 6 Z. Shen et al.

# 1 start5

|X-interval|(X1, V1)|(x1; VI)|(x2, V2)|3|
|---|---|---|---|---|
|Rectangle|Quadrilateral|1| | |
|2 1|(XZ,|(XL;| | |

# Coordinate

# Extra features

# BlockText

# BlockTypeReadingOrder

coordinatel
textblock1
textblock2
layout1
9
# A list of the layout elements

Fig. 2: The relationship between the three types of layout data structures.

Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such

## Working with Files

Many document loaders involve parsing files. The difference between such loaders usually stems from how the file is parsed, rather than how the file is loaded. For example, you can use `open` to read the binary content of either a PDF or a markdown file, but you need different parsing logic to convert that binary data into text.

As a result, it can be helpful to decouple the parsing logic from the loading logic, which makes it easier to re-use a given parser regardless of how the data was loaded.
You can use this strategy to analyze different files, with the same parsing parameters.

In [20]:
from patch_langchain_community.document_loaders.parsers import LlamaIndexPDFParser
from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path="./example_data/",
        glob="*.pdf",
    ),
    blob_parser=LlamaIndexPDFParser(),
)
docs = loader.load()
print(docs[0].page_content)
pprint.pp(docs[0].metadata)

# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis

Zejiang Shen1, Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain Lee4, Jacob Carlson3, and Weining Li5

1 Allen Institute for AI

shannons@allenai.org

2 Brown University

ruochen_zhang@brown.edu

3 Harvard University

melissadell@fas.harvard.edu,
jacob_carlson@fas.harvard.edu

4 University of Washington

bcgl@cs.washington.edu

5 University of Waterloo

w422li@uwaterloo.ca

# Abstract

Recent advances in document image analysis (DIA) have been primarily driven by the application of neural networks. Ideally, research outcomes could be easily deployed in production and extended for further investigation. However, various factors like loosely organized codebases and sophisticated model configurations complicate the easy reuse of important innovations by a wide audience. Though there have been ongoing efforts to improve reusability and simplify deep learning (DL) model development in disciplines like 

It is possible to work with files from cloud storage.

In [ ]:
from langchain_community.document_loaders import CloudBlobLoader
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader(
    blob_loader=CloudBlobLoader(
        url="s3:/mybucket", # Supports s3://, az://, gs://, file:// schemes.
        glob="*.pdf",
    ),
    blob_parser=LlamaIndexPDFParser(),
)
docs = loader.load()
print(docs[0].page_content)
pprint.pp(docs[0].metadata)

Find out more [here](/docs/how_to/document_loader_custom/)

## API reference

For detailed documentation of all `LlamaIndexPDFLoader` features and configurations head to the API reference: https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.LlamaIndexPDFLoader.html
